### 1. 在自制 RAG 框架中添加新 Embedding 模型并使用。
- 在huggingface找的新的模型jinaai/jina-embeddings-v2-base-z
- 修改前端代码 EmbeddingFile.jsx 加入新的变量  { value: 'jinaai/jina-embeddings-v2-base-zh', label: 'jina-embeddings-v2-base-zh' }
  ```
  huggingface: [
      { value: 'sentence-transformers/all-mpnet-base-v2', label: 'all-mpnet-base-v2' },
      { value: 'sentence-transformers/all-MiniLM-L6-v2', label: 'all-MiniLM-L6-v2' },
      { value: 'google-bert/bert-base-uncased', label: 'bert-base-uncased' },
      { value: 'jinaai/jina-embeddings-v2-base-zh', label: 'jina-embeddings-v2-base-zh' }
      
    ]
    ```
    运行截图
  ![运行截图](./2025-12-19_093503.png "运行截图")

In [1]:
import os
import json
from datetime import datetime
from services.loading_service import LoadingService
from services.chunking_service import ChunkingService
from services.embedding_service import EmbeddingService, EmbeddingConfig
from services.vector_store_service import VectorStoreService, VectorDBConfig
from services.search_service import SearchService
from services.parsing_service import ParsingService
import logging
from enum import Enum
from utils.config import VectorDBProvider
import pandas as pd
from pathlib import Path
from services.generation_service import GenerationService
from typing import List, Dict, Optional

/data/home/pi/anaconda3/envs/rag-project01/lib/python3.11/site-packages/pymilvus/client/__init__.py:6: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [ ]:
## 定义变量： model = 'jinaai/jina-embeddings-v2-base-zh' 是在huggingface找的新的模型

In [2]:
doc_id = 'DeepSeek-R1-技术报告中文版_pypdf_20250312225830.json'
provider = 'huggingface'
model = 'jinaai/jina-embeddings-v2-base-zh' 

In [3]:
doc_path = os.path.join("01-loaded-docs", doc_id)
with open(doc_path, 'r', encoding='utf-8') as f:
    doc_data = json.load(f)

In [4]:
doc_data

{'filename': 'DeepSeek-R1-技术报告中文版.pdf',
 'total_chunks': 5,
 'total_pages': 5,
 'loading_method': 'pypdf',
 'loading_strategy': None,
 'chunking_strategy': None,
 'chunking_method': 'loaded',
 'timestamp': '2025-03-12T22:58:30.376081',
 'chunks': [{'content': 'AIME 2024\n(Pass@1)Codeforces\n(Percentile)GPQA Diamond\n(Pass@1)MATH-500\n(Pass@1)MMLU\n(Pass@1)SWE-bench Verified\n(Resolved)020406080100\nAccuracy / Percentile (%)79.896.3\n71.597.3\n90.8\n49.279.296.6\n75.796.4\n91.8\n48.972.690.6\n62.194.3\n87.4\n36.863.693.4\n60.090.0\n85.2\n41.6\n39.258.7 59.190.2\n88.5\n42.0DeepSeek-R1 OpenAI-o1-1217 DeepSeek-R1-32B OpenAI-o1-mini DeepSeek-V3DeepSeek-R1 ：通过强化学习激励 LLMs 中的推理能力\n深度探索人 工 智 能\nresearch@deepseek.com\n摘要\n我们推出了第一代推理模型， DeepSeek-R1-Zero 和 DeepSeek-R1 。DeepSeek-R1-Zero 是一个通\n过大规模强化学习（ RL ）训练而成的模型，无需监督微调（ SFT ）作为初步步骤，展示了卓越\n的推理能力。通过  RL ，DeepSeek-R1-Zero 自然涌现出许多强大且有趣的推理行为。然而，它也\n面临诸如可读性差和语言混合等挑战。为了解决这些问题并进一步提升推理性能，我们推出了  \nDeepSeek-R1 ，它在 RL 之前引入了多阶段训练和冷启动数据。 DeepSeek-R1 在推理任务上实现\n

In [6]:
 # 创建 EmbeddingConfig 和 EmbeddingService
config = EmbeddingConfig(provider=provider, model_name=model)
embedding_service = EmbeddingService()
        
# 准备输入数据
input_data = {
            "chunks": doc_data["chunks"],
            "metadata": {
                "filename": doc_data["filename"],
                "total_chunks": doc_data["total_chunks"],
                "total_pages": doc_data["total_pages"],
                "loading_method": doc_data["loading_method"],
                "chunking_method": doc_data["chunking_method"]
            }
        }
        
# 创建嵌入 - 只接收两个返回值
embeddings, _ = embedding_service.create_embeddings(input_data, config)
        
# 保存嵌入结果
output_path = embedding_service.save_embeddings(doc_id, embeddings)

/data/home/pi/git/rag-project01-framework/backend/services/embedding_service.py:278: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  return HuggingFaceEmbeddings(
Some weights of BertModel were not initialized from the model checkpoint at jinaai/jina-embeddings-v2-base-zh and are newly initialized: ['embeddings.position_embeddings.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.dense.weight', 'encoder.layer.0.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.output.dense.bias', 'encoder.layer.0.output.dense.weight', 'encoder.layer.1.intermediate.dense.bias', 'encoder.layer.1.intermediate.

In [7]:
embeddings

[{'embedding': [-0.006207761820405722,
   -0.01615379936993122,
   0.009992249310016632,
   -0.025914808735251427,
   -0.023901522159576416,
   0.03374737128615379,
   -0.04018702730536461,
   0.0002037027443293482,
   -0.013204343616962433,
   -0.01946668140590191,
   -0.011068080551922321,
   -0.008991971611976624,
   0.015929104760289192,
   -0.022472916170954704,
   0.037511203438043594,
   0.03566156327724457,
   0.030912024900317192,
   0.0319417305290699,
   -0.06162500008940697,
   -0.02418545074760914,
   0.014794607646763325,
   0.0037423167377710342,
   0.0037184301763772964,
   0.03400115668773651,
   0.005996243096888065,
   -0.011242590844631195,
   -0.013598998077213764,
   -0.020339809358119965,
   0.0077074081636965275,
   -0.07544302940368652,
   0.02232511155307293,
   -0.004889479372650385,
   0.0028455695137381554,
   0.013526582159101963,
   0.019465165212750435,
   0.008822496049106121,
   -0.023609856143593788,
   -0.021374067291617393,
   -0.01956937462091446,


### 在自制 RAG 框架中添加 Chroma 并实现后端存储。
核心修改代码vector_store_service.py 增加存储方法根据 Vector Database 传入参数不同区分调用存储方法
```
def _index_to_chroma(self, embeddings_data: Dict[str, Any], config: VectorDBConfig) -> Dict[str, Any]:
        """
        将嵌入向量索引到 Chroma 数据库
        """
....
```
- 前端运行效果
- ![前端](./2025-12-19_094507.png "前端")

In [ ]:
## python 调用

In [8]:
file_id = 'DeepSeek-R1-技术报告中文版_huggingface_20251219012838.json'
vector_db = 'chroma'
index_mode = 'hnsw'

            
embedding_file = os.path.join("02-embedded-docs", file_id)

config = VectorDBConfig(provider=vector_db, index_mode=index_mode)
vector_store_service = VectorStoreService()
result = vector_store_service.index_embeddings(embedding_file, config)

In [9]:
result

{'database': 'chroma',
 'index_mode': 'hnsw',
 'total_vectors': 5,
 'index_size': 5,
 'processing_time': 0.088927,
 'collection_name': 'DeepSeek_R1_jishubaogaozhongwenban_huggingface_20251219015506'}